[View in Colaboratory](https://colab.research.google.com/github/Shreyas3108/Multipath-selection-using-ML/blob/master/TreeClass.ipynb)

In [0]:
!pip install -U -q PyDrive
 
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
 
# 1. Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [2]:
from google.colab import files

uploaded = files.upload()

for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))

Saving multipath.csv to multipath.csv
User uploaded file "multipath.csv" with length 25125562 bytes


In [4]:
!apt-get -qq install -y graphviz && pip install -q pydot
import pydot

Selecting previously unselected package fontconfig.
(Reading database ... 18298 files and directories currently installed.)
Preparing to unpack .../00-fontconfig_2.11.94-0ubuntu2_amd64.deb ...
Unpacking fontconfig (2.11.94-0ubuntu2) ...
Selecting previously unselected package libjbig0:amd64.
Preparing to unpack .../01-libjbig0_2.1-3.1_amd64.deb ...
Unpacking libjbig0:amd64 (2.1-3.1) ...
Selecting previously unselected package libcdt5.
Preparing to unpack .../02-libcdt5_2.38.0-16ubuntu2_amd64.deb ...
Unpacking libcdt5 (2.38.0-16ubuntu2) ...
Selecting previously unselected package libcgraph6.
Preparing to unpack .../03-libcgraph6_2.38.0-16ubuntu2_amd64.deb ...
Unpacking libcgraph6 (2.38.0-16ubuntu2) ...
Selecting previously unselected package libtiff5:amd64.
Preparing to unpack .../04-libtiff5_4.0.8-5ubuntu0.1_amd64.deb ...
Unpacking libtiff5:amd64 (4.0.8-5ubuntu0.1) ...
Selecting previously unselected package libwebp6:amd64.
Preparing to unpack .../05-libwebp6_0.6.0-3_amd64.deb ...
Unpa

Setting up libltdl7:amd64 (2.4.6-2) ...
Setting up libxcb-shm0:amd64 (1.12-1ubuntu1) ...
Setting up libxpm4:amd64 (1:3.5.12-1) ...
Setting up libxt6:amd64 (1:1.1.5-1) ...
Setting up libthai-data (0.1.26-3) ...
Setting up libcdt5 (2.38.0-16ubuntu2) ...
Setting up fontconfig (2.11.94-0ubuntu2) ...
Regenerating fonts cache... done.
Setting up libcgraph6 (2.38.0-16ubuntu2) ...
Setting up libwebp6:amd64 (0.6.0-3) ...
Setting up libcairo2:amd64 (1.14.10-1ubuntu1) ...
Setting up libgvpr2 (2.38.0-16ubuntu2) ...
Setting up libgd3:amd64 (2.2.5-3) ...
Setting up libharfbuzz0b:amd64 (1.4.2-1) ...
Setting up libthai0:amd64 (0.1.26-3) ...
Setting up libxmu6:amd64 (2:1.1.2-2) ...
Setting up libpango-1.0-0:amd64 (1.40.12-1) ...
Setting up libxaw7:amd64 (2:1.0.13-1) ...
Setting up libpangoft2-1.0-0:amd64 (1.40.12-1) ...
Setting up libpangocairo-1.0-0:amd64 (1.40.12-1) ...
Setting up libgvc6 (2.38.0-16ubuntu2) ...
Setting up graphviz (2.38.0-16ubuntu2) ...
Processing triggers for libc-bin (2.26-0ubuntu2

In [5]:
!ls

datalab  multipath.csv


In [0]:
import pandas as pd 
import numpy as np 

In [0]:
data = pd.read_csv('multipath.csv')

In [8]:
data.head()

,name,time,EncodingPath,Producer,bandwidth,input-data-rate,input-load,input-packet-rate,interface-name,load-interval,output-data-rate,output-load,output-packet-rate,reliability,peak-input-data-rate,peak-output-data-rate,peak-input-packet-rate,peak-output-packet-rate,delay,infname
0,Cisco-IOS-XR-infra-statsd-oper:infra-statistic...,1501867007733000000,Cisco-IOS-XR-infra-statsd-oper:infra-statistic...,leaf1,100000000.0,7214502.0,18.0,639356.0,HundredGigE0/0/0/4,0.0,12225031.0,31.0,1080487.0,255.0,0.0,0.0,0.0,0.0,0,True
1,Cisco-IOS-XR-infra-statsd-oper:infra-statistic...,1501867007735000000,Cisco-IOS-XR-infra-statsd-oper:infra-statistic...,leaf1,100000000.0,6564805.0,16.0,598775.0,HundredGigE0/0/0/5,0.0,11132639.0,28.0,980453.0,255.0,0.0,0.0,0.0,0.0,7,True
2,Cisco-IOS-XR-infra-statsd-oper:infra-statistic...,1501867007738000000,Cisco-IOS-XR-infra-statsd-oper:infra-statistic...,leaf1,100000000.0,13838151.0,35.0,1217949.0,HundredGigE0/0/0/6,0.0,9664580.0,24.0,856189.0,255.0,0.0,0.0,0.0,0.0,3,True
3,Cisco-IOS-XR-infra-statsd-oper:infra-statistic...,1501867007740000000,Cisco-IOS-XR-infra-statsd-oper:infra-statistic...,leaf1,100000000.0,10752468.0,27.0,968818.0,HundredGigE0/0/0/7,0.0,13502426.0,34.0,1146930.0,255.0,0.0,0.0,0.0,0.0,0,True
4,Cisco-IOS-XR-infra-statsd-oper:infra-statistic...,1501867007742000000,Cisco-IOS-XR-infra-statsd-oper:infra-statistic...,leaf1,100000000.0,6925863.0,17.0,605078.0,HundredGigE0/0/0/8,0.0,12163551.0,31.0,1055221.0,255.0,0.0,0.0,0.0,0.0,8,True


In [0]:
from sklearn.preprocessing import LabelEncoder

In [0]:
le = LabelEncoder()
data['path'] = le.fit_transform(data['interface-name'])

In [0]:
data = pd.get_dummies(data , columns= ['Producer'])

In [12]:
data.head()

,name,time,EncodingPath,bandwidth,input-data-rate,input-load,input-packet-rate,interface-name,load-interval,output-data-rate,...,Producer_leaf3,Producer_leaf4,Producer_leaf5,Producer_leaf6,Producer_leaf7,Producer_leaf8,Producer_spine1,Producer_spine2,Producer_spine3,Producer_spine4
0,Cisco-IOS-XR-infra-statsd-oper:infra-statistic...,1501867007733000000,Cisco-IOS-XR-infra-statsd-oper:infra-statistic...,100000000.0,7214502.0,18.0,639356.0,HundredGigE0/0/0/4,0.0,12225031.0,...,0,0,0,0,0,0,0,0,0,0
1,Cisco-IOS-XR-infra-statsd-oper:infra-statistic...,1501867007735000000,Cisco-IOS-XR-infra-statsd-oper:infra-statistic...,100000000.0,6564805.0,16.0,598775.0,HundredGigE0/0/0/5,0.0,11132639.0,...,0,0,0,0,0,0,0,0,0,0
2,Cisco-IOS-XR-infra-statsd-oper:infra-statistic...,1501867007738000000,Cisco-IOS-XR-infra-statsd-oper:infra-statistic...,100000000.0,13838151.0,35.0,1217949.0,HundredGigE0/0/0/6,0.0,9664580.0,...,0,0,0,0,0,0,0,0,0,0
3,Cisco-IOS-XR-infra-statsd-oper:infra-statistic...,1501867007740000000,Cisco-IOS-XR-infra-statsd-oper:infra-statistic...,100000000.0,10752468.0,27.0,968818.0,HundredGigE0/0/0/7,0.0,13502426.0,...,0,0,0,0,0,0,0,0,0,0
4,Cisco-IOS-XR-infra-statsd-oper:infra-statistic...,1501867007742000000,Cisco-IOS-XR-infra-statsd-oper:infra-statistic...,100000000.0,6925863.0,17.0,605078.0,HundredGigE0/0/0/8,0.0,12163551.0,...,0,0,0,0,0,0,0,0,0,0


In [0]:

inp = data[['bandwidth','input-data-rate','input-load','load-interval','output-data-rate','output-load',
            'output-packet-rate','reliability','peak-input-data-rate','peak-output-data-rate'
           ,'peak-input-packet-rate', 'peak-output-packet-rate', 'delay','Producer_leaf1', 
            'Producer_leaf2', 'Producer_leaf3', 'Producer_leaf4','Producer_leaf5',
            'Producer_leaf6', 'Producer_leaf7', 'Producer_leaf8',
           'Producer_spine1', 'Producer_spine2', 'Producer_spine3',
           'Producer_spine4']] 
out = data['path']

In [14]:
from sklearn.cross_validation import train_test_split
x_train , y_train , x_test , y_test = train_test_split(inp , out , test_size = 0.3)

/usr/local/lib/python2.7/dist-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [15]:
from sklearn.tree import DecisionTreeClassifier 
from sklearn.metrics import accuracy_score

mod = DecisionTreeClassifier() 

mod.fit(x_train , x_test) 
pred = mod.predict(y_train) 
accuracy_score(y_test , pred)

0.7728852838933952

In [47]:
mod = DecisionTreeClassifier(criterion = 'entropy' , max_depth = 400) 

mod.fit(x_train , x_test) 
pred = mod.predict(y_train) 
accuracy_score(y_test , pred)

0.7731335871544446

In [0]:
from sklearn.ensemble import RandomForestClassifier

In [41]:
mod2 = RandomForestClassifier()
mod2.fit(x_train , x_test)
pred2 = mod2.predict(y_train)
accuracy_score(y_test , pred2)

0.7912597252110578

In [44]:
mod2 = RandomForestClassifier(criterion = 'entropy' ,n_estimators = 20 , max_depth= 400 , max_features= 7)
mod2.fit(x_train , x_test)
pred2 = mod2.predict(y_train)
accuracy_score(y_test , pred2)

0.8026816752193345

In [63]:
mod2 = RandomForestClassifier(n_estimators = 20 , max_depth = 400 , max_features = 8 )
mod2.fit(x_train , x_test)
pred2 = mod2.predict(y_train)
accuracy_score(y_test , pred2)

0.8052888594603542

In [0]:
from sklearn.ensemble import ExtraTreesClassifier

In [69]:
mod3 = ExtraTreesClassifier(n_estimators= 25 , max_depth= 200 , max_features = 8 )
mod3.fit(x_train, x_test)
pred3 = mod3.predict(y_train)
accuracy_score(y_test , pred3)

0.803136897864592

In [76]:
mod3 = ExtraTreesClassifier(criterion = 'entropy' , n_estimators = 25 , max_features = 16 , max_depth = 400)
mod3.fit(x_train, x_test)
pred3 = mod3.predict(y_train)
accuracy_score(y_test , pred3)

0.8073166694255918

In [77]:
mod3 = ExtraTreesClassifier(n_estimators= 25 , max_depth= 400 , max_features = 16 )
mod3.fit(x_train, x_test)
pred3 = mod3.predict(y_train)
accuracy_score(y_test , pred3)

0.8088892567455719

In [0]:
from sklearn.svm import SVC

In [0]:
svc = SVC()
svc.fit(x_train ,x_test)
pred4 = svc.predict(y_train)
accuracy_score(y_test , pred4)

In [0]:
!pip install keras
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasClassifier
from keras.utils import np_utils
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.preprocessing import LabelEncoder
from sklearn.pipeline import Pipeline